# This notebook is for Coursera Capstone Project

In [1]:
import pandas as pd
import numpy as np

In [2]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


# Part 1: Obtain and preprocess the data

### Scrape the wikipedia page and transform the table into a pandas dataframe

In [3]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]

### Replace the string "Not assigned " to numpy.NaN

In [4]:
df.replace(to_replace='Not assigned', value=np.NaN, inplace=True)

### Drop the rows with a borough that is "not assigned"

In [5]:
df.dropna(subset=['Borough'],inplace=True)

### For those cells with no neighbourhood names, use the borough names

In [6]:
df.loc[df['Neighbourhood'].isna(), 'Neighbourhood'] = df.loc[df['Neighbourhood'].isna()].loc[:,'Borough']

### Combine the rows with same Postcode and Borough

In [7]:
df = df.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(lambda x: ','.join(map(str, x))).reset_index()

### The shape of dataframe is 103 * 3

In [8]:
df.shape

(103, 3)

# Part 2: Query the  geographical coordinates

### Try the geocoder way but failed

In [9]:
!pip install geocoder
import geocoder

In [10]:
postcodes = df['Postcode'].to_list()
lat = []
lng = []

### HTTPS connection error

In [11]:
lat_lng_coords = None
for postcode in postcodes:
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postcode))
        lat_lng_coords = g.latlng
    print('get 1 record!')
    lat.append(lat_lng_coords[0])
    lng.append(lat_lbg_coords[1])

KeyboardInterrupt: 

### Instead, try to read the csv to get the coordinates

In [12]:
df_coords = pd.read_csv('http://cocl.us/Geospatial_data')

### Join two dataframes to get the lat and lng coordinate data

In [13]:
df = df.join(df_coords.set_index('Postal Code'), on='Postcode')

In [14]:
df

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848


# Part 3: Explore and cluster the neighborhood

In [15]:
import folium
import requests
from geopy.geocoders import Nominatim

### Visualize all neighborhoods on the map

In [18]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [19]:
# create map of Tronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Define Foursquare credentials and version

In [20]:
CLIENT_ID = 'NFGFBGEWVP2LCF5R3EQY0NOWPHNU3B5ROB4BS2RS0UDB5TTN' # your Foursquare ID
CLIENT_SECRET = '3Y2T5DIG0IRDZZHM0L3D5I2EG5YJKI25ZJUFMMSN45DXUD5J' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

### Define a function to repeatly get 100 venues for each neighborhood

In [21]:
LIMIT = 100

In [22]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Run the code above on each neighborhood and store the venue data into the dataframe toronto_venues

In [23]:
toronto_venues = getNearbyVenues(df['Neighbourhood'], df['Latitude'], df['Longitude'])

Rouge,Malvern
Highland Creek,Rouge Hill,Port Union
Guildwood,Morningside,West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park,Ionview,Kennedy Park
Clairlea,Golden Mile,Oakridge
Cliffcrest,Cliffside,Scarborough Village West
Birch Cliff,Cliffside West
Dorset Park,Scarborough Town Centre,Wexford Heights
Maryvale,Wexford
Agincourt
Clarks Corners,Sullivan,Tam O'Shanter
Agincourt North,L'Amoreaux East,Milliken,Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview,Henry Farm,Oriole
Bayview Village
Silver Hills,York Mills
Newtonbrook,Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park,Don Mills South
Bathurst Manor,Downsview North,Wilson Heights
Northwood Park,York University
CFB Toronto,Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens,Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West,Riverdale
The Beaches West,Indi

Check the size of the resulting dataframe

In [24]:
print(toronto_venues.shape)
toronto_venues.head()

(2271, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge,Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
3,"Guildwood,Morningside,West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant


### Only 100 neighborhoods have venues data, not all of 103

In [25]:
len(toronto_venues['Neighborhood'].unique())

100

### Delete those 3 rows without any venues data from the original dataframe

In [26]:
df_index_with_venues_data = pd.DataFrame(toronto_venues['Neighborhood'].unique())
df_index_with_venues_data

,0
0,"Rouge,Malvern"
1,"Highland Creek,Rouge Hill,Port Union"
2,"Guildwood,Morningside,West Hill"
3,Woburn
4,Cedarbrae
5,Scarborough Village
6,"East Birchmount Park,Ionview,Kennedy Park"
7,"Clairlea,Golden Mile,Oakridge"
8,"Cliffcrest,Cliffside,Scarborough Village West"
9,"Birch Cliff,Cliffside West"


In [27]:
df = df.join(df_index_with_venues_data.set_index(0), on='Neighbourhood', how='inner').reset_index()

In [28]:
df.drop(columns=['index'], inplace=True)

In [29]:
df

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848


Check how many venues were returned for each neighborhood

In [30]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,King,Richmond",100,100,100,100,100,100
Agincourt,4,4,4,4,4,4
"Agincourt North,L'Amoreaux East,Milliken,Steeles East",3,3,3,3,3,3
"Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown",10,10,10,10,10,10
"Alderwood,Long Branch",9,9,9,9,9,9
"Bathurst Manor,Downsview North,Wilson Heights",18,18,18,18,18,18
Bayview Village,4,4,4,4,4,4
"Bedford Park,Lawrence Manor East",25,25,25,25,25,25
Berczy Park,55,55,55,55,55,55


Check out how many unique categories can be curated from all the returned venues

In [31]:
len(toronto_venues['Venue Category'].unique())

273

### Onehot encode the toronto_venues

In [32]:
toronto_onehot = pd.get_dummies(toronto_venues['Venue Category'])
toronto_onehot.shape
toronto_onehot['Neighbourhood'] = toronto_venues['Neighborhood']

#Move the column 'Neighbourhood' to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

In [33]:
toronto_onehot.head()

,Neighbourhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Rouge,Malvern",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Highland Creek,Rouge Hill,Port Union",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Guildwood,Morningside,West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Guildwood,Morningside,West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Guildwood,Morningside,West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [34]:
toronto_grouped = toronto_onehot.groupby(['Neighbourhood']).mean().reset_index()
toronto_grouped.head()

,Neighbourhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide,King,Richmond",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.03,...,0.0,0.02,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.0
1,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
4,"Alderwood,Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0


In [35]:
#confirm the size
toronto_grouped.shape

(100, 274)

### Create a new dataframe to and display the top 10 venues for each neighborhood

In [36]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [37]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Café,Bar,Steakhouse,Cosmetics Shop,Hotel,Restaurant,Burger Joint,American Restaurant,Thai Restaurant
1,Agincourt,Lounge,Breakfast Spot,Skating Rink,Clothing Store,Yoga Studio,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",Park,Playground,Sculpture Garden,Yoga Studio,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",Grocery Store,Fast Food Restaurant,Liquor Store,Fried Chicken Joint,Sandwich Place,Coffee Shop,Beer Store,Pizza Place,Pharmacy,Electronics Store
4,"Alderwood,Long Branch",Pizza Place,Gym,Dance Studio,Pharmacy,Coffee Shop,Skating Rink,Sandwich Place,Pub,Yoga Studio,Dog Run


### Cluster neighborhoods

In [38]:
from sklearn.cluster import KMeans

In [39]:
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

kmeans = KMeans(n_clusters=kclusters).fit(toronto_grouped_clustering)

kmeans.labels_

array([0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0,
       0, 4, 0, 0, 0, 0, 2, 0, 2, 0, 1, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0,
       0, 2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 2])

In [40]:
df.insert(3, 'Cluster Label', kmeans.labels_)

In [41]:
df.head()

,Postcode,Borough,Neighbourhood,Cluster Label,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",0,43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",0,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",2,43.763573,-79.188711
3,M1G,Scarborough,Woburn,0,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,0,43.773136,-79.239476


### Visualize the resulting clusters

In [42]:
import matplotlib.cm as cm
import matplotlib.colors as colors

In [43]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df['Latitude'], df['Longitude'], df['Neighbourhood'], df['Cluster Label']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Clusters

#### Join two dataframes first

In [44]:
toronto_merged = df.join(neighborhoods_venues_sorted.set_index('Neighbourhood'),on='Neighbourhood').reset_index()

In [45]:
toronto_merged.head()

,index,Postcode,Borough,Neighbourhood,Cluster Label,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,M1B,Scarborough,"Rouge,Malvern",0,43.806686,-79.194353,Fast Food Restaurant,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Hardware Store
1,1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",0,43.784535,-79.160497,Bar,Yoga Studio,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Festival
2,2,M1E,Scarborough,"Guildwood,Morningside,West Hill",2,43.763573,-79.188711,Intersection,Breakfast Spot,Pizza Place,Rental Car Location,Electronics Store,Medical Center,Mexican Restaurant,Donut Shop,Diner,Discount Store
3,3,M1G,Scarborough,Woburn,0,43.770992,-79.216917,Coffee Shop,Korean Restaurant,Yoga Studio,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant
4,4,M1H,Scarborough,Cedarbrae,0,43.773136,-79.239476,Bakery,Hakka Restaurant,Bank,Caribbean Restaurant,Athletics & Sports,Fried Chicken Joint,Thai Restaurant,Donut Shop,Discount Store,Dog Run


#### Cluster 1

In [46]:
toronto_merged.loc[toronto_merged['Cluster Label'] == 0, toronto_merged.columns[[3] + list(range(7, toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Rouge,Malvern",Fast Food Restaurant,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Hardware Store
1,"Highland Creek,Rouge Hill,Port Union",Bar,Yoga Studio,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Festival
3,Woburn,Coffee Shop,Korean Restaurant,Yoga Studio,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant
4,Cedarbrae,Bakery,Hakka Restaurant,Bank,Caribbean Restaurant,Athletics & Sports,Fried Chicken Joint,Thai Restaurant,Donut Shop,Discount Store,Dog Run
5,Scarborough Village,Playground,Grocery Store,Yoga Studio,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
6,"East Birchmount Park,Ionview,Kennedy Park",Discount Store,Department Store,Hobby Shop,Convenience Store,Coffee Shop,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore
7,"Clairlea,Golden Mile,Oakridge",Bakery,Fast Food Restaurant,Intersection,Bus Station,Bus Line,Metro Station,Soccer Field,Park,Gift Shop,Dessert Shop
8,"Cliffcrest,Cliffside,Scarborough Village West",American Restaurant,Movie Theater,Motel,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Falafel Restaurant
9,"Birch Cliff,Cliffside West",College Stadium,Skating Rink,General Entertainment,Café,Empanada Restaurant,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Dessert Shop
10,"Dorset Park,Scarborough Town Centre,Wexford He...",Indian Restaurant,Vietnamese Restaurant,Light Rail Station,Latin American Restaurant,Chinese Restaurant,Pet Store,Convenience Store,Discount Store,Falafel Restaurant,Event Space


#### Cluster 2

In [47]:
toronto_merged.loc[toronto_merged['Cluster Label'] == 1, toronto_merged.columns[[3] + list(range(7, toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
76,"Brockton,Exhibition Place,Parkdale Village",Coffee Shop,Breakfast Spot,Café,Climbing Gym,Stadium,Burrito Place,Restaurant,Caribbean Restaurant,Pet Store,Bakery


#### Cluster 3

In [48]:
toronto_merged.loc[toronto_merged['Cluster Label'] == 2, toronto_merged.columns[[3] + list(range(7, toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,"Guildwood,Morningside,West Hill",Intersection,Breakfast Spot,Pizza Place,Rental Car Location,Electronics Store,Medical Center,Mexican Restaurant,Donut Shop,Diner,Discount Store
16,Hillcrest Village,Fast Food Restaurant,Dog Run,Mediterranean Restaurant,Golf Course,Pool,Drugstore,Diner,Discount Store,Doner Restaurant,Donut Shop
63,"The Annex,North Midtown,Yorkville",Coffee Shop,Café,Sandwich Place,Pizza Place,American Restaurant,Indian Restaurant,Pub,BBQ Joint,Jewish Restaurant,Pharmacy
72,Caledonia-Fairbanks,Park,Women's Store,Market,Fast Food Restaurant,Yoga Studio,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop
74,"Dovercourt Village,Dufferin",Supermarket,Pharmacy,Bakery,Pet Store,Art Gallery,Café,Middle Eastern Restaurant,Bar,Bank,Music Venue
80,"High Park,The Junction South",Café,Mexican Restaurant,Bakery,Thai Restaurant,Italian Restaurant,Bar,Discount Store,Cajun / Creole Restaurant,Fried Chicken Joint,Speakeasy
89,"Humber Bay,King's Mill Park,Kingsway Park Sout...",Business Service,Baseball Field,Pool,Yoga Studio,Drugstore,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
93,Humber Summit,Pizza Place,Empanada Restaurant,Yoga Studio,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant
99,Northwest,Rental Car Location,Bar,Drugstore,Yoga Studio,Dim Sum Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant


#### Cluster 4

In [49]:
toronto_merged.loc[toronto_merged['Cluster Label'] == 3, toronto_merged.columns[[3] + list(range(7, toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
53,St. James Town,Coffee Shop,Café,Hotel,Restaurant,Italian Restaurant,Cocktail Bar,Bakery,Cosmetics Shop,Clothing Store,Breakfast Spot


#### Cluster 5

In [50]:
toronto_merged.loc[toronto_merged['Cluster Label'] == 4, toronto_merged.columns[[3] + list(range(7, toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
67,Stn A PO Boxes 25 The Esplanade,Coffee Shop,Italian Restaurant,Restaurant,Café,Hotel,Beer Bar,Seafood Restaurant,Cocktail Bar,Breakfast Spot,Park
